In [1]:
import pandas as pd


In [2]:
df = pd.read_csv('titles.csv')

df = df[['description', 'genres']]

df = df[df.genres != '[]']

df

,description,genres
0,This collection includes 12 World War II-era p...,['documentation']
1,A mentally unstable Vietnam War veteran works ...,"['drama', 'crime']"
2,Intent on seeing the Cahulawassee River before...,"['drama', 'action', 'thriller', 'european']"
3,"King Arthur, accompanied by his squire, recrui...","['fantasy', 'action', 'comedy']"
4,12 American military prisoners in World War II...,"['war', 'action']"
...,...,...
5843,The story follows a young man and woman who go...,"['romance', 'comedy', 'drama']"
5845,A beautiful love story that can happen between...,"['romance', 'drama']"
5846,A heart warming film that explores the concept...,['drama']
5847,A controversial TV host and comedian who has b...,['comedy']


In [3]:
puncs = ['[',']',"'",' ']

for punc in puncs:

    df['genres'] = df['genres'].str.replace(punc,'')

df['genres'] = df['genres'].str.split(',')

df.head(10)

/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  """


,description,genres
0,This collection includes 12 World War II-era p...,[documentation]
1,A mentally unstable Vietnam War veteran works ...,"[drama, crime]"
2,Intent on seeing the Cahulawassee River before...,"[drama, action, thriller, european]"
3,"King Arthur, accompanied by his squire, recrui...","[fantasy, action, comedy]"
4,12 American military prisoners in World War II...,"[war, action]"
5,A British sketch comedy series with the shows ...,"[comedy, european]"
6,"Brian Cohen is an average young Jewish man, bu...",[comedy]
7,When a madman dubbed 'Scorpio' terrorizes San ...,"[thriller, action, crime]"
8,"In the 1930s, bored waitress Bonnie Parker fal...","[crime, drama, action]"
9,Two small children and a ship's cook survive a...,"[romance, action, drama]"


In [4]:
df

,description,genres
0,This collection includes 12 World War II-era p...,[documentation]
1,A mentally unstable Vietnam War veteran works ...,"[drama, crime]"
2,Intent on seeing the Cahulawassee River before...,"[drama, action, thriller, european]"
3,"King Arthur, accompanied by his squire, recrui...","[fantasy, action, comedy]"
4,12 American military prisoners in World War II...,"[war, action]"
...,...,...
5843,The story follows a young man and woman who go...,"[romance, comedy, drama]"
5845,A beautiful love story that can happen between...,"[romance, drama]"
5846,A heart warming film that explores the concept...,[drama]
5847,A controversial TV host and comedian who has b...,[comedy]


In [5]:


df[df['genres'].apply(type) != list]


,description,genres


In [6]:
df[df['genres'].apply(len) < 1]

,description,genres


In [7]:
df

,description,genres
0,This collection includes 12 World War II-era p...,[documentation]
1,A mentally unstable Vietnam War veteran works ...,"[drama, crime]"
2,Intent on seeing the Cahulawassee River before...,"[drama, action, thriller, european]"
3,"King Arthur, accompanied by his squire, recrui...","[fantasy, action, comedy]"
4,12 American military prisoners in World War II...,"[war, action]"
...,...,...
5843,The story follows a young man and woman who go...,"[romance, comedy, drama]"
5845,A beautiful love story that can happen between...,"[romance, drama]"
5846,A heart warming film that explores the concept...,[drama]
5847,A controversial TV host and comedian who has b...,[comedy]


In [9]:
gen_list = df['genres'].explode('genres')

gen_set = set(gen_list.to_list())

gen_set

{'action',
 'animation',
 'comedy',
 'crime',
 'documentation',
 'drama',
 'european',
 'family',
 'fantasy',
 'history',
 'horror',
 'music',
 'reality',
 'romance',
 'scifi',
 'sport',
 'thriller',
 'war',
 'western'}

In [ ]:
# check data types
print(df.description.apply(type))
print(df.genres.apply(type))

In [ ]:
df = df['genres'].astype(list)

df.genres.apply(type)

In [ ]:
df['genres'].apply(len)

In [ ]:
# create a sample dataframe
df = pd.DataFrame({'fruits': [['apple', 'banana'], [], ['apple', 'cherry']]})
df

In [ ]:
# filter out rows with empty lists
df = df[df['fruits'].apply(len) > 0]

df 